In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import joblib

In [3]:
df1=pd.read_csv('C:\\Users\\lenovo\\Desktop\\STRUCTURAL-HEALTH-MONITORING\\Data-SHM-Br 1-704 (data1).csv')
df2=pd.read_csv('C:\\Users\\lenovo\\Desktop\\STRUCTURAL-HEALTH-MONITORING\\Data-SHM-Br 1-704 (data2).csv')
df3=pd.read_csv('C:\\Users\\lenovo\\Desktop\\STRUCTURAL-HEALTH-MONITORING\\Data-SHM-Br 1-704 (data3).csv') 
df4=pd.read_csv('C:\\Users\\lenovo\\Desktop\\STRUCTURAL-HEALTH-MONITORING\\Data-SHM-Br 1-704 (data4).csv')
df5=pd.read_csv('C:\\Users\\lenovo\\Desktop\\STRUCTURAL-HEALTH-MONITORING\\Data-SHM-Br 1-704 (data5).csv')
df6=pd.read_csv('C:\\Users\\lenovo\\Desktop\\STRUCTURAL-HEALTH-MONITORING\\Data-SHM-Br 1-704 (data6).csv')
df7=pd.read_csv('C:\\Users\\lenovo\\Desktop\\STRUCTURAL-HEALTH-MONITORING\\Data-SHM-Br 1-704 (data7).csv')


In [4]:
df1.head(10)

,TIMESTAMP,RECORD,VOLTAGE,ST1,ST2,ST3
0,2022-01-11 14:15:14.22,0,12.60,2172.691,791.9388,-3759.116
1,2022-01-11 14:15:14.23,1,12.60,2173.406,792.1773,-3758.128
2,2022-01-11 14:15:14.24,2,12.61,2172.588,792.5861,-3759.388
3,2022-01-11 14:15:14.25,3,12.60,2173.474,792.5861,-3758.366
4,2022-01-11 14:15:14.26,4,12.60,2173.099,794.7662,-3757.242
5,2022-01-11 14:15:14.27,5,12.60,2174.939,794.4937,-3755.573
6,2022-01-11 14:15:14.28,6,12.60,2174.632,796.2310,-3754.313
7,2022-01-11 14:15:14.29,7,12.59,2176.472,795.3112,-3754.006
8,2022-01-11 14:15:14.3,8,12.61,2175.961,797.6277,-3752.644
9,2022-01-11 14:15:14.31,9,12.60,2178.005,798.7518,-3750.974


In [5]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)

In [6]:
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], errors='coerce')  # convert to datetime
df['hour'] = df['TIMESTAMP'].dt.hour

In [7]:
df.head(5)

,TIMESTAMP,RECORD,VOLTAGE,ST1,ST2,ST3,hour
0,2022-01-11 14:15:14.220,0,12.60,2172.691,791.9388,-3759.116,14.0
1,2022-01-11 14:15:14.230,1,12.60,2173.406,792.1773,-3758.128,14.0
2,2022-01-11 14:15:14.240,2,12.61,2172.588,792.5861,-3759.388,14.0
3,2022-01-11 14:15:14.250,3,12.60,2173.474,792.5861,-3758.366,14.0
4,2022-01-11 14:15:14.260,4,12.60,2173.099,794.7662,-3757.242,14.0


In [8]:
for col in ['VOLTAGE', 'ST1', 'ST2', 'ST3']:
    max_row = df.loc[df[col].idxmax()]
    print(f"Max {col}: {max_row[col]} at {max_row['TIMESTAMP']}")


Max VOLTAGE: 12.76 at 2022-01-23 17:22:30.970000
Max ST1: 2294.424 at 2022-01-22 07:39:23.010000
Max ST2: 1070.338 at 2022-01-22 07:39:23.020000
Max ST3: -3196.344 at NaT


In [9]:
# Keep only records between 7 AM and 12 PM
df_filtered = df[(df['hour'] >= 7) & (df['hour'] <= 24)]


In [10]:
print(df.shape)
print(df_filtered.shape)

(114459467, 7)
(78387673, 7)


In [11]:
X=df_filtered[['VOLTAGE','ST1','ST2','ST3']]

In [12]:
X.head(5)

,VOLTAGE,ST1,ST2,ST3
0,12.60,2172.691,791.9388,-3759.116
1,12.60,2173.406,792.1773,-3758.128
2,12.61,2172.588,792.5861,-3759.388
3,12.60,2173.474,792.5861,-3758.366
4,12.60,2173.099,794.7662,-3757.242


In [13]:
X_scaled = StandardScaler().fit_transform(X)

In [14]:
# Build Autoencoder
input_dim = X_scaled.shape[1]
encoding_dim = 2  

input_layer = Input(shape=(input_dim,))
encoded = Dense(8, activation='relu')(input_layer)
encoded = Dense(encoding_dim, activation='relu')(encoded)

decoded = Dense(8, activation='relu')(encoded)
decoded = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
encoder = Model(inputs=input_layer, outputs=encoded)

autoencoder.compile(optimizer=Adam(0.001), loss='mse', )
autoencoder.fit(X_scaled, X_scaled, epochs=20, batch_size=10000, shuffle=True)

Epoch 1/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.1880
Epoch 2/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.0414
Epoch 3/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.0410
Epoch 4/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.0407
Epoch 5/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - loss: 0.0400
Epoch 6/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - loss: 0.0394
Epoch 7/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.0390
Epoch 8/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - loss: 0.0386
Epoch 9/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 795s 2ms/step - loss: 0.0370
Epoch 10/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.0362
Epoch 11/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.0358
Epoch 12/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.0354
Epoch 13/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - loss: 0.0344
Epoch 14/20
7839/7839 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.0341
Epoch 15/20
78

In [15]:
X_scaled.shape

(78387673, 4)

# Step 1: Get row indices

# Step 2: Use them to slice actual data

In [16]:
# Encode data
X_=np.random.choice(X_scaled.shape[0],size=500000,replace=False) 
X_sample = X_scaled[X_]
X_encoded = encoder.predict(X_sample)

15625/15625 ━━━━━━━━━━━━━━━━━━━━ 8s 510us/step


In [17]:
#clustering the data
model=KMeans(n_clusters=3, random_state=42)

In [ ]:
df_sample=df.iloc[X_]

from sklearn.cluster import MiniBatchKMeans

model = MiniBatchKMeans(n_clusters=3, batch_size=10000, random_state=42)
df_sample['Cluster'] = model.fit_predict(X_sample)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_18516\3753673513.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['Cluster'] = model.fit_predict(X_sample)


saving the sample dataset with labels

In [30]:
df_sample.to_csv('sampled_500k.csv', index=False)

In [31]:
sampled_df = pd.read_csv('sampled_500k.csv')
sampled_df.head(5)

,TIMESTAMP,RECORD,VOLTAGE,ST1,ST2,ST3,hour,Cluster
0,2022-01-18 13:09:54.800,5304382,12.28,2130.772,863.4481,-3420.173,13.0,0
1,2022-01-19 05:19:42.830,11123185,12.16,2170.084,906.9361,-3372.200,5.0,0
2,2022-01-18 16:31:05.760,6511478,12.26,2137.063,873.3318,-3407.026,16.0,0
3,2022-01-12 08:27:26.480,6553226,12.39,2185.281,901.8490,-3612.984,8.0,2
4,2022-01-12 05:55:07.430,5639321,12.39,2193.305,913.3926,-3606.686,5.0,2


In [20]:
print(df_sample.groupby('Cluster')[['VOLTAGE', 'ST1', 'ST2', 'ST3']].mean())


           VOLTAGE          ST1         ST2          ST3
Cluster                                                 
0        12.236427  2159.407395  896.342199 -3414.404351
1        12.186012  2164.029456  899.409450 -3420.237370
2        12.405873  2177.912634  893.238491 -3605.603433


Sample prediction

In [28]:
print(model.predict([[14.9, 2100, 1000, -4999]]))

[2]


In [29]:
model_filename = "bridge_health_model1.pkl"
joblib.dump(model, model_filename)

['bridge_health_model1.pkl']